In [2]:
import sys
sys.path.append("..")
sys.path.append("../src")

import threading, time, requests, json
import uvicorn
from asxai.services.search.search_api import app
from asxai.services.search.search_workers import start_search_engine

Kafka broker up and ready


In [29]:
import requests

API_URL = "http://localhost:8080"
user_id = "jul"
notebook_id = "my-notebook"
query = "what is the role of the amygdala according to Girardeau as from her papers published between 2016 and 2025?"

In [31]:
payload1 = {
    "user_id": user_id,
    "notebook_id": notebook_id,
    "query": query
}
res = requests.post(f"{API_URL}/search", json=payload1)
res.json()
# res = requests.post(f"{API_URL}/search", json=payload1)
# res.json()
# res = requests.post(f"{API_URL}/search", json=payload2)
# res.json()

{'task_id': '5c022ffb48a537cbf1e0/47f210eacb89414efd82'}

In [16]:
# res = requests.get(f"{API_URL}/search/user/jul/notebooks/test-notebook").json()
res = requests.get(f"{API_URL}/search/5c022ffb48a537cbf1e0/47f210eacb89414efd82").json()
res

{'task_id': '5c022ffb48a537cbf1e0/47f210eacb89414efd82',
 'notebook': {'id': '5c022ffb48a537cbf1e0/47f210eacb89414efd82',
  'task_id': '5c022ffb48a537cbf1e0/47f210eacb89414efd82',
  'query': 'what is the role of the amygdala according to Girardeau as from her papers published between 2016 and 2025?',
  'query_hash': '5d4f231df016694c7c15da4f9c581e9d4da037bad55e9b80af20a5259cdd8bb8',
  'user_id': 'jul',
  'notebook_id': 'my-notebook',
  'qdrant_model': 'intfloat/multilingual-e5-large-instruct',
  'rerank_model': '1-2025-05-18',
  'blend_weights': {},
  'result': [{'paperIdQ': '3abb2e9f-e737-5659-9671-d81700d39a56',
    'influentialCitationCount': 0.0,
    'publicationDate': '2025-04-25',
    'paperId': 'ad68b16410200b91448dc2bc40656db3dab519de',
    'authorName': 'JF Morici,I. Paiva-Lima,A. Silva,É. Pronier,G. Girardeau',
    'venue': 'bioRxiv',
    'fieldsOfStudy': 'Biology',
    'openAccessPdf': 'https://www.biorxiv.org/content/biorxiv/early/2024/06/13/2024.06.13.598918.full.pdf',
   

In [4]:
res2['result']['parsed_query']

KeyError: 'result'

In [115]:
[(r['qdrant_score'], r['user_score']) for r in res['result']['result']]

TypeError: 'Response' object is not subscriptable

In [1]:
import sys
sys.path.append("..")
sys.path.append("../src")

import config
from asxai.dataIO import load_data
from asxai.vectorDB import QdrantManager
from asxai.vectorDB import PaperEmbed

In [2]:
embedEngine = PaperEmbed(model_name=config.MODEL_EMBED)
qdrant = QdrantManager()

INFO:asxai.vectorDB.embed:CUDA available: True


In [3]:
query = "what defines a species?"

In [4]:
query_emb = embedEngine.embed_queries([query])

In [5]:
query_emb

tensor([[ 0.6260, -0.2159, -1.2148,  ..., -1.2656, -1.0459, -0.5898]],
       dtype=torch.float16)

In [6]:
len(query_emb[0])

1024

In [7]:
res = await qdrant.client.get_collection(qdrant.collection_name_ids)
print(res.config.params.vectors.size)

1024


In [9]:
res = await qdrant.query_batch_streamed(query_vectors=query_emb.tolist(), topK=20, payload_filters=[[('title','any',['Dynamic']), ('authorName', 'any', ['Steg', 'Mravinac']), ('publicationDate', 'lte', '2025-02-05')]], 
                                        with_vectors=False, collection_name=qdrant.collection_name_ids)

In [12]:
res[0].points[0].payload['publicationDate']

'2025-02-05'

In [9]:
res = await qdrant.query_batch_streamed(query_vectors=query_emb.tolist(), topK=50, filter_payloads=[['publicationDate', 'gt', '2025-01-01'],['paperId','==',['0027ea27f5986f9c6ee9776e10289c7f2aa089ca','a1ea5b916228bd88b391567eb2d57a5bf02dc6ff']]], with_vectors=False, collection_name=qdrant.collection_name_chunks)

In [24]:
res[0].groups[1].hits[1].payload['paperId']

'a1ea5b916228bd88b391567eb2d57a5bf02dc6ff'

In [27]:
best_chunks = {}
for qid in res.keys():
    best_texts = {}
    for Q_groups in res[qid].groups:
        paperId = Q_groups.hits[0].payload['paperId']
        best_texts[paperId] = [Q_groups.hits[k].payload['text'] for k in range(len(Q_groups.hits))]
    best_chunks[qid] = best_texts

In [28]:
best_chunks

{0: {'0027ea27f5986f9c6ee9776e10289c7f2aa089ca': ['Generalized compiler support for store speculation that guarantees the correct order of poisoning is the key challenge that we solve in this paper. 3 Background In this section, we describe the architectural support needed to enable speculative DAE and some compiler preliminaries. Our speculation technique requires architectural support for predicated stores and FIFOs. Store values are tagged with a poison bit that, when set, causes the corresponding store request to be dropped in the DU without committing a store. We say that a store request gets killed (or poisoned) if its corresponding store value has the poison bit set. This is a lightweight form of speculation that does not require replays in the CU and does not result in out-of-bounds stores, be- cause mis-speculated stores are never committed. Speculative loads can be supported by simply discarding the value of a mis-speculated load. Predicated stores are easy to support in hard

In [ ]:
res[0].points[0].payload | {"score_qdrant": res[0].points[0].score}

{'paperIdQ': '6465b1c5-8000-51d5-9669-96fb75109ffd',
 'paperId': '0027ea27f5986f9c6ee9776e10289c7f2aa089ca',
 'citationCount': 1,
 'abstract': 'None',
 'publicationYear': 2025,
 'influentialCitationCount': 0.0,
 'openAccessPdf': 'gs://arxiv-dataset/arxiv/arxiv/pdf/2501/2501.13553v1.pdf',
 'authorName': 'Robert Szafarczyk,S. Nabi,W. Vanderbauwhede',
 'venue': 'International Conference on Compiler Construction',
 'main_text': 'Compiler Support for Speculation in Decoupled Access/Execute Architectures Syed Waqar Nabi University of Glasgow Glasgow, United Kingdom syed.nabi@glasgow.ac.uk Robert Szafarczyk University of Glasgow Glasgow, United Kingdom robert.szafarczyk@glasgow.ac.uk Wim Vanderbauwhede University of Glasgow Glasgow, United Kingdom wim.vanderbauwhede@glasgow.ac.uk Abstract Irregular codes are bottlenecked by memory and communi- cation latency. Decoupled access/execute (DAE) is a common technique to tackle this problem. It relies on the compiler to separate memory address gener

In [8]:
res = await qdrant.query_batch_streamed(query_vectors=query_emb.tolist(), topK=50, offset=50, with_vectors=True, collection_name=qdrant.collection_name_ids)
res[0].points[0]

ScoredPoint(id='d0aa7e18-5e0c-57ad-ab2e-5c4b355b5b3e', version=4337, score=0.79184455, payload={'paperIdQ': 'd0aa7e18-5e0c-57ad-ab2e-5c4b355b5b3e', 'paperId': '82696e684a577ca208c782b39c1285b53643a26a', 'citationCount': 0, 'abstract': 'None', 'publicationYear': 2025, 'influentialCitationCount': 0.0, 'openAccessPdf': 'https://figshare.com/articles/journal_contribution/Importance_of_the_integrated_use_of_bioacoustics_and_mist_nets_for_bat_surveys/28450887/1/files/52498398.pdf', 'authorName': 'Jeanneson Silva de Sales,Hannah Larissa Nunes de Lima,F. L. Rocha,P. Cordeiro-Estrela,Patrício Adriano da Rocha', 'venue': 'Bioacoustics', 'main_text': 'Supplementary  Figure  1.  Accumulation  curves  for  bat  species,  based  on  data  from acoustic  monitoring  of  five  forest  fragments,  from  the  Metropolitan  Region  of  João Pessoa/PB,  northeastern  Brazil.  a)  Overall  Forest  Fragments;  b)  Floresta  Nacional  da Restinga de Cabedelo; c) Jardim Florestal Municipal da Cidade Verde; d)

In [ ]:
res = await qdrant.query_batch_streamed(query_vectors=[query_emb.tolist()], topK=100, filter_payloads=[['publicationDate', 'gt', '2025-01-01']], with_vectors=True, collection_name=qdrant.collection_name_ids)
print(res[0].groups[1].hits[0].payload['title'])
res = await qdrant.query_batch_streamed(query_vectors=[res[0].groups[1].hits[0].vector], topK=100, offset=100, filter_payloads=[['publicationDate', 'gt', '2025-01-01']], with_vectors=True, collection_name=qdrant.collection_name_ids)

In [ ]:
res

In [ ]:
match_titles = [res[0].groups[k].lookup.payload['title'] for k in range(len(res[0].groups))]
[print(title) for title in match_titles];

In [28]:
match_titles = [res[0].groups[k].lookup.payload['title'] for k in range(len(res[0].groups))]
[print(title) for title in match_titles];

Land use system effects on biodiversity of litter ants (Hymenoptera: Formicidae) in tropical rainforest, South Region of Cameroon
Species Delimitation and Cryptic Diversity in Rheotanytarsus Thienemann & Bause, 1913 (Diptera: Chironomidae) Based on DNA Barcoding
DNA barcoding reveals mislabeling of endangered sharks sold as swordfish in New England fish markets
Taxonomic implications of leaf anatomy in Thai Smilax (Smilacaceae)
Disentangling the Taxonomy of Parides burchellanus (Lepidoptera: Papilionidae) and Related Taxa.
TaxDiff: taxonomic-guided diffusion model for protein sequence generation
Where are my nematodes? labelling and visualising entomopathogenic nematodes in vivo using carbon quantum dots.
FACdb: a comprehensive resource for genes, gut microbiota, and metabolites in farm animals
Adaptive Divergence and Functional Convergence: The Evolution of Pulmonary Gene Expression in Amphibians of the Qingzang Plateau.
The birds of the archaeological sites of the Acropolis area (Ath

In [29]:
import nest_asyncio
nest_asyncio.apply()

ref_ids = []
for k in range(6, 4, -1):
    if not ref_ids:
        res = await qdrant.query_batch_streamed(query_vectors=query_emb[0][k:k+1].tolist(), vector_name=f"embed_{k}", topK=50, filter_payloads=[['publicationDate', 'gt', '2025-02-01']])
    else:
        res = await qdrant.query_batch_streamed(query_vectors=query_emb[0][k:k+1].tolist(), vector_name=f"embed_{k}", topK=50, filter_payloads=[['paperId', 'in', ref_ids]])
    ref_ids = [res[0].groups[k].lookup.payload['paperId'] for k in range(len(res[0].groups))]
    match_titles = [res[0].groups[k].lookup.payload['title'] for k in range(len(res[0].groups))]
    match_texts = [res[0].groups[k].lookup.payload['main_text'] for k in range(len(res[0].groups))]
    if not ref_ids:
        ref_titles = match_titles
    # print(" ")

[print(title) for title in match_titles];

Where are my nematodes? labelling and visualising entomopathogenic nematodes in vivo using carbon quantum dots.
Taxonomic implications of leaf anatomy in Thai Smilax (Smilacaceae)
The Spiky Side of Urban Wildlife, First Detection of a Zoonotic Assemblage of Giardia duodenalis in European Hedgehogs (Erinaceus europaeus) from Italy
Dscamb regulates cone mosaic formation in zebrafish via filopodium-mediated homotypic recognition
Land use system effects on biodiversity of litter ants (Hymenoptera: Formicidae) in tropical rainforest, South Region of Cameroon
Erratum: DAVID L. EMERY, NATHAN J. EMERY & LINDSAY W. POPPLE (2025) Description of three new cicada species in the genus Yoyetta Moulds (Hemiptera: Cicadidae: Cicadettinae), from eastern Australia. Zootaxa, 5590 (2): 151-184.
Importance of the integrated use of bioacoustics and mist nets for bat surveys
Two new species of Crassitegula (Sebdeniaceae, Sebdeniales, Rhodophyta) extend the diversity and biogeographic range of the genus to th

In [72]:
res[0].groups[k].lookup.payload.keys()

dict_keys(['paperIdQ', 'paperId', 'publicationDate', 'publicationYear', 'citationCount', 'venue', 'fieldsOfStudy', 'openAccessPdf', 'abstract', 'title', 'influentialCitationCount', 'authorName', 'main_text'])

In [31]:
[print(title) for title in match_titles];

Land use system effects on biodiversity of litter ants (Hymenoptera: Formicidae) in tropical rainforest, South Region of Cameroon
Species Delimitation and Cryptic Diversity in Rheotanytarsus Thienemann & Bause, 1913 (Diptera: Chironomidae) Based on DNA Barcoding
Sigmodontine rodent diversity: The Frankenstein paradox
Comparison of growth, relative abundance, and diet of three sympatric Hemiandrus ground wētā (Orthoptera, Anostostomatidae) in a New Zealand Forest
Species and site contributions to Odonata adult assemblage uniqueness in Brazilian Amazon streams
DNA barcoding reveals mislabeling of endangered sharks sold as swordfish in New England fish markets
Taxonomic implications of leaf anatomy in Thai Smilax (Smilacaceae)
TAXONOMY OF THE KATYDIDS (ORTHOPTERA: TETTIGONIIDAE) FROM EAST ASIA AND ADJACENT ISLANDS. COMMUNICATION 17
Disentangling the Taxonomy of Parides burchellanus (Lepidoptera: Papilionidae) and Related Taxa.
From virtue to sin: Is the installation of bat boxes an effect

In [7]:
from transformers import pipeline
import torch

# model_id = "unsloth/Mistral-Small-24B-Instruct-2501-unsloth-bnb-4bit"
model_id = "unsloth/mistral-7b-instruct-v0.3-bnb-4bit"
pipe = pipeline(
    "text-generation",
    model=model_id)

`low_cpu_mem_usage` was None, now default to True since model is quantized.
Device set to use cuda:0


In [10]:
articles = '\n\nabstract#'.join(f"{k}\n" + txt for k, txt in enumerate(match_texts[2:3]))
query="What is the role of amygdala?"
messages = [
    {"role": "system", "content": """
Summarize the article below to help answer a question. 
Do not directly answer the question, instead summarize the evidence present in the excerpt that
help answer the question. 
Reply ’Not applicable’ if the excerpt is irrelevant. 
At the end of your response, provide a score from 1-10 on a newline indicating the relevance
of the text to the question. 
DO NOT EXPLAIN YOUR SCORE.
     """},
    {"role": "user", "content": f"""
Question: {query}
Article: 
{articles}

Relevant Information Summary:"""}
]
# query="What is the role of amygdala?"
# messages = f"""<s>[SYSTEM_PROMPT]Summarize the article below to help answer a question. 
# Do not directly answer the question, instead summarize the evidence present in the excerpt that
# help answer the question. 
# Reply ’Not applicable’ if the excerpt is irrelevant. 
# At the end of your response, provide a score from 1-10 on a newline indicating the relevance
# of the text to the question. 
# Do not explain your score.[/SYSTEM_PROMPT]

# [INST]
# Question: {query}
# Article: 
# {articles}

# Relevant Information Summary:[/INST]"""

outputs = pipe(
    messages,
    max_new_tokens=10000,
)
# print(outputs[0]["generated_text"][-1]['content'])

In [11]:
print(outputs[0]["generated_text"][-1]['content'])

 The article discusses the role of dynorphin-expressing neurons in the central amygdala (CeADyn) in alcohol drinking. The study used fiber photometry to record real-time activity of these neurons in male and female mice during voluntary alcohol consumption, water consumption, and sucrose consumption. The results showed that there is a unique functional signature for alcohol in the CeADyn neuron population, with a large increase in calcium transients after bouts of licking for alcohol, and only modest increases during licking for water or 0.5% sucrose. This increase in activity was present even when controlling for differences in drinking behavior unique to alcohol, such as longer bout durations. The study suggests that understanding the specific brain mechanisms underlying the behavioral and physiological responses to alcohol that promote alcohol misuse is important for understanding the role of these neurons in alcohol drinking.

Relevance Score: 10

Explanation: The article provides 

In [37]:
print(outputs[0]["generated_text"].split("[/INST]")[-1])

The article discusses a study that investigates the activity of dynorphin-expressing neurons in the central amygdala (CeADyn) during alcohol drinking in mice. Key findings include:

- CeADyn neurons show a large increase in calcium transients after bouts of licking for alcohol, compared to modest increases for water or sucrose.
- The unique engagement of CeADyn neurons during alcohol consumption is not fully explained by differences in drinking behavior, such as longer bout durations for alcohol.
- The study used fiber photometry to record neuronal activity and multilevel modeling to analyze the data, revealing significant effects of time and solution type on neuronal activity.
- The study also explored the relationship between drinking characteristics (such as bout duration and temporal occurrence) and neuronal activity, finding that while these characteristics modulate signal, they do not fully explain the unique alcohol-evoked CeADyn activity.
- The study suggests that the unique ph

In [19]:
print(messages)

<s>[SYSTEM_PROMPT]You are a research assistant. Answer the user’s query as accurately as possible using your knowledge and the provided articles.     Do not summarize the abstracts unless explicitly asked. Use your knowledge first if the question is general. Try as much as possible to cite the provided abstract which are      referenced as abstract#1, abstract#2, etc[/SYSTEM_PROMPT]     [INST]Here are the abstracts of recent papers:

0
AmygdalaGo-BOLT3D: A boundary learning transformer for tracing human amygdala 1Center for Brain Imaging Science and Technology, Zhejiang University, Hangzhou, 310027, Zhejiang, China. 2College of Biomedical Engineering and Instrument Science, Zhejiang University, Hangzhou, 310027, China. 3State Key Laboratory of Cognitive Neuroscience and Learning, Beijing Normal University, Beijing, 100875, China. 4Faculty of Psychology, Beijing Normal University, Beijing, 100875, Beijing, China. 5School of Psychology, Capital Normal University, Beijing, 100190, China. 